# Студенческий модуль

Этот набор кода выполнит за Вас тяжёлую работу по считыванию файлов заданий и расчёту регрессионных моделей, оставив Вам самую лёгкую часть: выбор гиперпараметров и интерпретацию результатов. Запускайте ячейки по очереди.

In [ ]:
# Если следующие строки выполняются с ошибками, проверяйте, как Вы ставили sklearn и Jupyter.
import numpy as np
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [ ]:
# Пожалуйста, положите выданные Вам файлы рядом с этим файлом и укажите ниже их имена:
X_train = np.loadtxt('train_X_03.txt')
y_train = np.loadtxt('train_y_03.txt')
X_test = np.loadtxt('test_X_03.txt')
# Если при выполнении этой ячейки происходит OSError, убедитесь, что файлы лежат в той же папке,
# а также что Вы вводите их имена правильно.

In [ ]:
# Попробуем нарисовать все спектры, чтобы посмотреть, с каким набором данных мы имеем дело
plt.plot(X_train.T); plt.show()

In [ ]:
# Теоретически, максимальное возможное количество компонентов равно наименьшей
# из размерностей матрицы спектров, но мы начнём с меньшего их количества.
max_ncomp = 16

# Рассчитаем среднеквадратическую ошибку при перекрёстной проверке 
# индивидуально для каждого компонента
cv = [
    GridSearchCV(
        PLSRegression(scale = False),
        {'n_components': list(range(1, max_ncomp+1))},
        scoring = 'neg_root_mean_squared_error',
        return_train_score = True,
        cv=5
    ).fit(X_train, y_train[:,i])
    for i in range(y_train.shape[1])
]
# Извлечём результаты и сохраним их в виде матриц
ncomps = np.array([
    [p['n_components'] for p in v.cv_results_['params']] for v in cv
]).T
RMSEP = np.array([
    -v.cv_results_['mean_train_score'] for v in cv
]).T
RMSECV = np.array([
    -v.cv_results_['mean_test_score'] for v in cv
]).T

In [ ]:
plt.subplot(1, 2, 1); plt.plot(ncomps, RMSEP);
plt.subplot(1, 2, 2); plt.plot(ncomps, RMSECV); plt.legend(list(range(1, y_train.shape[1]+1))); plt.show()

In [ ]:
# Для столбцов [1, 2, 3, 4, 5] я выбираю ... компонент:
# Замените 1 на число компонент.
n_comps = (1, 1, 1, 1, 1)

Обучим модели с выбранными количествами компонент, а также нарисуем графики весов PLS (матрицы $\mathbf{W}$):

In [ ]:
models = []
for i in range(len(n_comps)):
    m = PLSRegression(
        n_components = n_comps[i], scale = False
    ).fit(X_train, y_train[:,i])
    models.append(m)
    plt.subplot(1, len(n_comps), i+1)
    plt.plot(m.x_weights_)
plt.show()
    
predictions = []
for i in range(len(n_comps)):
    predictions.append(models[i].predict(X_test)[:,0])

np.savetxt('y_test.txt', np.array(predictions).T)

Если Вы дошли до этого момента, у Вас должен был получиться файл с предсказанными концентрациями, `y_test.txt`. Отправьте его на <mailto:ikrylov@laser.chem.msu.ru>. Туда же пришлите графики матриц весов и объяснения, почему Вы считаете, что данные количества компонент являются оптимальными.

Если что-то не получается, пожалуйста, обращайтесь на тот же адрес, я отвечу.